# UT Austin CS 370 Undergraduate Research Project Spring 2022
## Researcher: Tejas Saboo <br> Supervisor: Professor Angela Beasley

# Machine Learning Model for the Stylistic Classification of Fine Art (Multi-Class)

### Introduction

Emerging technologies are disrupting the art industry and changing the way we create and experience art. As a computer science, mathematics, and statistics student with a deep appreciation for arts and culture, I am fascinated by how technology shapes our society. One recent technological advancement I am excited about is OpenAI's Dall-E 2, an AI system that can create realistic images and art from a description in natural language. I enjoy learning about innovative inventions and discoveries, and I love applying my learnings to build impactful technologies. For my undergraduate research project, I decided to explore the intersection of art, design, and technology to identify and address opportunities to augment human creativity. The project entailed architecting a deep convolutional neural network model to classify visual fine artwork by its artistic style.

### Results
The final machine learning model presented is a deep convolutional neural network for the multi-class classification of fine art by its artistic style. The style of an artwork refers to its distinctive visual elements, techniques, and methods, typically corresponding with an art movement or school group that the artist is associated with. The model below can be used for any binary classification problem, and the overall model architecture can be used for multi-class classification with minor modifications. Currently, the model is used to distinguish between the artistic styles of Cubism, Ukiyo-e, Northern Renaissance, and Pointillism. The model is highly successful with a 97.1250% accuracy on the training set and 46.0000% accuracy on the test set after 70 epochs of training.


### Experimental Process
I experimented with a variety of different technical approaches and model architectures throughout this project. First, I reviewed neural networks and machine learning concepts, learned about computer vision, and determined the relevant libraries. I decided that using Python's PyTorch library would be ideal for creating a customizable and scalable model for image classification tasks. Then, I scoped out my project and established a timeline of major milestones. Next, I searched for art databases and found a dataset with images of flower and marina paintings. After finding a relevant dataset, I set up a custom data loader using the torchvision library's ImageFolder and torch library's DataLoader functions. Then, I performed a manual train-test split. For each stylistic category, I moved 80 images into the training set and 20 images into the test set. Next, I designed a simple linear neural network model, created a training function, and implemented an evaluation function. I trained the model on the flower and marina painting dataset, then performed hyperparameter tuning. Then, I realized that the model was too simple, so I experimented with various architectures such as linear and non-linear neural networks, convolutional neural networks, fully convolutional neural networks, and deep convolutional neural networks. By limiting the size of the initial train and test sets, I was able to quickly train the models and continuously prototype the model architecture. Next, I searched for a dataset with artwork categorized by its artistic style, and I discovered the Wikiart database and found a corresponding Wikiart dataset that I decided to use for my final model. I updated the model to be compatible with the wikiart dataset and retrained it. However, I realized that manually moving images from the wikiart directory to the train and test directories would be tedious and impractical at scale, so I decided to automate the process. I learned how to use the shutil.move functionality to automate moving files from one directory to another, resulting in a scalable train-test split that also randomly shuffles the images. This allowed me to have train and test datasets on the order of hundreds and thousands of images in seconds. Then, I retrained the model on the larger dataset for a binary classification problem. Noting the low training accuracy, I updated the model to use blocks. While this improved the training accuracy significantly, I now had the problem of overfitting since there was a significant disparity between the training and testing accuracies. I implemented data augmentation in the training image transform function, experimented with various criterion loss functions, tested stochastic gradient descent versus adam for the optimizer, investigated regularization techniques, added dropouts, and tried various learning rates throughout my hyperparameter tuning process. This successfully reduced overfitting and improved model train and test accuracy. Then, I modified the model for multi-class image classification tasks. Finally, I modularized my functions, added documentation, and made final improvements. 

### Future Plans
My future plans to build on this project include experimenting with computer vision to extract visual features and elements of the artwork such as line, shape, texture, form, space, color, value, composition, perspective, and subject matter to create algorithms that suggest tags for artwork. Next, I will create an algorithm that generates original and relevant titles for the artwork using the tags and extracted visual features. I will also investigate recent artificial intelligence and machine learning advancements to create an algorithm that measures the creativity and craftsmanship expemplified in the artwork and provides actionable feedback to help artists improve. Furthermore, I plan on learning more about blockchain and NFTs and possibly launching my own project.

### Involvement in the Scientific Community
Throughout the semester, I made a conscious effort to become more involved and engaged with the scientific community. My first step was to learn and contribute to academia through this undergraduate research project. Next, I read research publications and articles about technological advancements in industry. I decided to take this a step further by attending research conferences and symposiums, faculty talks, industry events, and honors thesis defenses. These include:
<br>
- Institute for Foundations of Machine Learning: AI for Accurate and Fair Imaging with Alex Dimakis
<br>
- Good Systems Annual Symposium: Defining, Evaluating, and Building Ethical AI Systems
<br>
- 2022 UT Machine Learning Laboratory Research Symposium
<br>
- Quantum Computing Triple Play at UT Austin (Quantum Conference with Dr. Aaronson, Atom Computing, and Strangeworks)
<br>
- Tesla Cyber Rodeo
<br> 
- Chat with Jahmy Hindman, the CTO of John Deere
<br>
- Bloomberg Engineering: Inclusive Branch (Re)Naming at Scale
<br>
- Hosting UT Entrepreneurship Week and Moderating the Democratizing Healthcare Innovation Panel
<br>
- Undergraduate Honors Thesis Defense
<br>
- Quantum Information Science Final Presentations

### Dataset

Downloadable Dataset Link: https://drive.google.com/file/d/182-pFiKvXPB25DbTfAYjJ6gDE-ZCRXz0/view

The dataset used for this model is scraped from Wikiart.org, a visual art encyclopedia. The dataset has several images corresponding to various artistic styles. In the dataset, there are several sub-directories. The name of each sub-directory is the artistic style, and every image of a painting within those sub-directories belong to that given artistic style category. 

### Imports

In [1]:
# Imports for moving files and ML functionality
import os
import random
import shutil
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

### Functions to Automate the Train-Test Dataset Split

The move_shuffled_files function moves a random subset of files from the source directory to the destination directory.

The train_test_split function automates the process of moving files from the original wikiart directory to the train and test directories for every relevant artistic style category. I decided to use an 80:20 split because a research publication by Gholamy et al., Why 70/30 or 80/20 Relation Between Training and Testing Sets: A Pedagogical Explanation, found that empirical evidence suggests that using 80% of data for training and the remaining 20% of data for testing causes the best results for smaller datasets (Gholamy et al., 2018). 

In [2]:
# Randomly selects num_files images from the src directory and moves them to the dest directory
def move_shuffled_files(src, dest, num_files):
    # Get the list of all files in the src directory
    files = os.listdir(src)
    # Select a random sample of num_files from files and move them to the dest directory
    for file_name in random.sample(files, num_files):
        shutil.move(os.path.join(src, file_name), dest)

In [3]:
# Automates the train-test split to move images from the wikiart dataset to corresponding train and test directories
def train_test_split(wiki_dirs, train_size, test_size):
    # Initialize the base directories
    base_src = 'wikiart/'
    base_train_dest = 'data/train/'
    base_test_dest = 'data/test/'
    # Perform a train-test split for each desired art category
    for wiki_dir in wiki_dirs:
        # Get the src for the current art category
        src = base_src + wiki_dir
        # Get the train and test dests for the current art category
        train_dest = base_train_dest + wiki_dir
        test_dest = base_test_dest + wiki_dir
        # Perform a train-test split by moving files from src to the train and test dest directories
        move_shuffled_files(src, train_dest, train_size)
        move_shuffled_files(src, test_dest, test_size)

In [4]:
# art_wiki_directories is the set of desired art categories to classify 
art_wiki_directories = ['Cubism', 'Ukiyo_e', 'Northern_Renaissance', 'Pointillism']
# Perform a 80:20 train-test split on the desired art categories 
train_test_split(art_wiki_directories, 200, 50)

### Train and Test Transform Functions To Set Up Images for the Model

The train and test transform functions resize the images to 256 x 256 x 3 and convert them to tensors. This allows the data to be efficiently computed in the neural network. I experimented with several sizes ranging from 32 x 32 x 3 to 512 x 512 x 3, and I found that 256 x 256 x 3 resulted in the optimal time-accuracy trade off. Furthermore, I implemented data augmentation through the RandomHorizontalFlip, ColorJitter, and RandomGrayscale transformations in the train transform function to reduce overfitting. During hyperparameter tuning, I also tested several different probabilities for the parameters of these functions. 

In [5]:
# Transform function for the training data with data augmentation to reduce overfitting
train_transform = transforms.Compose([
                                      transforms.Resize([256, 256]),
                                      transforms.RandomHorizontalFlip(p=0.4),
                                      transforms.ColorJitter(0.5,0.5,0.5,0.5),
                                      transforms.RandomGrayscale(0.10),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5))
])

# Transform function for the test data
test_transform = transforms.Compose([
                                      transforms.Resize([256, 256]),
                                      transforms.ToTensor(),
])

### Generate Train and Test Sets

The following 2 cells generate the train and test sets for the model. They utilize the above transformation functions and some library functionality. The datasets.ImageFolder method in torchvision's library facilitates the process of accessing images from desired folders while maintaining information about their corresponding class labels. The PyTorch DataLoader creates an iterable dataset designed specifically for neural networks. Shuffle is set to true in the train data loader to permute the indices of all samples to prevent overfitting. Additionally, a batch size of 4 was selected through hyperparameter tuning to reduce memory overhead and overfitting.

In [6]:
# Set up the train set using ImageFolder and DataLoader
train_data = datasets.ImageFolder(root="data/train", transform=train_transform)
train = DataLoader(train_data, batch_size = 4, shuffle = True)

In [7]:
# Set up the test set using ImageFolder and DataLoader
test_data = datasets.ImageFolder(root="data/test", transform=test_transform)
test = DataLoader(test_data, batch_size = 4, shuffle = False)

### Deep Convolutional Neural Network

I created a deep convolutional neural network for the fine art classification task. This model is ideal because it provides an efficient and dense network with a strong performance in image classification. The deep convolutional neural network architecture helps the model detect low-level features in the initial layers and high-level features in the subsequent layers, resulting in a robust model with the ability to classify complex images. The model accomplishes this by taking advantage of the 2D structure of input images using local connections and tied weights followed by polling to translate invariant features. Furthermore, this architecture allows the model to have fewer parameters than fully connected networks with the same number of hidden units, resulting in faster training. Additionally, my design leverages blocks, which are components with multiple layers, to increase model complexity without limiting readability. The model has 3 blocks, and each block has a series of batch normalizations, activation functions, convolutonal layers, and max pooling. Finally, I employ several strategies to reduce overfitting, such as the use of data augmentation in the train transform function, applying batch normalization and max pooling, and using dropouts. I also performed hyperparameter tuning on the model architecture and number of blocks, activation functions, dropout probabilities, data agumentation transformation functions and their probabilities, batch sizes, image resize dimensions, learning rate, criterion loss functions, and stochastic gradient descent versus adam optimizers, among others. 

In [8]:
# Block structure for the neural network
class CNNBlock(torch.nn.Module):
    
    def __init__(self, in_channels, out_channels, conv_kernel, conv_stride, conv_padding, pool_kernel, pool_stride):
        super().__init__()
        # L is the set of layers for the block
        L = [
            # Applies batch normalization on the input channels
            torch.nn.BatchNorm2d(in_channels),
            # Applies the rectified linear unit activation function
            torch.nn.ReLU(),
            # Applies a 2D convolutional layer
            torch.nn.Conv2d(in_channels, out_channels, conv_kernel, conv_stride, conv_padding, dilation=1, 
                            groups=1, bias=True, padding_mode='zeros', device=None, dtype=None),
            # Applies batch normalization on the output channels
            torch.nn.BatchNorm2d(out_channels),
            # Applies randomized leaky rectified linear unit activation function
            torch.nn.RReLU(0.1, 0.25),
            # Applies a 2D max pooling over an input signal composed of several input planes
            torch.nn.MaxPool2d(pool_kernel, pool_stride),
        ]
        # self.network holds a sequential container of the layers defined above
        self.network = torch.nn.Sequential(*L)
    
    # Perform forward propagation on the block
    def forward(self, x):
        # Forward propagate the input through the sequential layers
        return self.network(x)

# Deep Convolutional Neural Network Classifier Model for Binary Classification 
class DeepCNNClassifier(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        # in_channels is the number of channels in the input image
        in_channels = 3
        # out_channels is the number of channels produced by the convolution
        out_channels = 32
        # num_classes is the number of art categories being classified
        num_classes = 4
        # kernel_size is the size of the filter mask window
        kernel_size = 7
        # stride is the size of the movement of the mask window over the image
        stride = 1
        # padding is the amount of pixels being added to the image
        padding = 3
        # L is the set of layers for the CNN
        L = [
            # Add a CNN Block
            CNNBlock(in_channels, out_channels, kernel_size, stride, padding, 2, 2),
            # Randomly zeroes elements of the input tensor with 0.2 probability to reduce overfitting
            torch.nn.Dropout(p=0.25),
            # Add a CNN Block
            CNNBlock(32, 64, 3, 1, 1, 2, 2),
            # Randomly zeroes elements of the input tensor with 0.25 probability to reduce overfitting
            torch.nn.Dropout(p=0.25),
            # Add a CNN Block
            CNNBlock(64, 128, 3, 1, 1, 2, 2),
            # Applies a 2D max pooling over an input signal composed of several input planes
            torch.nn.MaxPool2d(2),
            # Creates a feature vector by flattening data into a 1D tensor 
            torch.nn.Flatten(),
            # Applies a linear transformation for classification
            torch.nn.Linear(32768, num_classes)
        ]
        # self.network holds a sequential container of the layers defined above
        self.network = torch.nn.Sequential(*L)
        # self.transforms holds a transform function to normalize data
        self.transforms = torchvision.transforms.Compose(
            [torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    # Perform forward propagation on the CNN
    def forward(self, x):
        # Apply transform to normalize input
        x = self.transforms(x)
        # Forward propagate the input through the sequential layers
        return self.network(x)

### Set Up the Model

The below cell initializes the model and its criterion loss function and optimizer for back propagation. The cross entropy loss function computes the cross entropy loss between the input and target. The adam optimizer implements the Adam method, which is a computationally efficient algorithm for first-order gradient-descent based optimization of stochastic objective functions based on adaptive estimates of lower-order movements. Extensive hyperparameter tuning resulted in the use of cross entropy loss as the criterion, Adam as the optimizer, and 0.001 as the learning rate.

In [9]:
# Initalize the Deep Convolutional Neural Network Model
model = DeepCNNClassifier()
# Criterion loss function for training the neural network
criterion = nn.CrossEntropyLoss()
# Optimizer for modifying weights during back propagation
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

### Set Up Training Functionality

The train_iter function trains the model for one epoch, and the train_model function runs the train_iter function for the desired number of epochs in a loop. Training the model entails getting predictions from the model for all training input images, computing the loss between the predicted label and the true label for all training images, and performing back propagation to fine-tune and update the model weights to improve performance.

In [10]:
# Train the model for one epoch
def train_iter(epoch, net, trainDataLoader, optimizer, criterion):
    # Set the model in training mode
    net.train()
    # Initialize counters
    total = 0
    train_loss = 0
    correct = 0
    # Train on every image input and target classification label pair from the train set
    for inputs, targets in trainDataLoader:
        # Set the optimizer to zero grad to make it non-cumulative by zeroing out all gradients
        optimizer.zero_grad()
        # Get predictions from the model for the inputs
        outputs = net(inputs)
        # Get the (value, index) tuples for the maximum values across the 1st dimension of outputs
        _, predicted = torch.max(outputs.data, 1)
        # Compute the loss which is the distance of the prediction from the true label
        loss = criterion(outputs, targets)
        # Perform back propagation to update weights
        loss.backward()
        # Gradient descent
        optimizer.step()
        # Update loss, total, and correct counters
        train_loss += loss.item()
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
    
    # Compute and print the average loss and accuracy
    avg_loss = train_loss / len(trainDataLoader)
    accuracy = 100 * correct / total
    print('Epoch: {} \tTraining Loss: {:.25f} \tTraining Accuracy: {:.4f} %'.format(epoch, avg_loss, accuracy))
    return accuracy

In [11]:
# Train the model for the desired number of epochs, num_epochs
def train_model(num_epochs, net, trainDataLoader, optimizer, criterion, acc_thresh):
  # Call the train_iter function for every epoch
  for i in range(num_epochs):
    acc = train_iter(i, net, trainDataLoader, optimizer, criterion)
    # Early stopping to avoid overfitting and stop training after reaching desired training accuracy
    if acc >= acc_thresh:
        break

### Train the Model

In [12]:
# Train the model
train_model(50, model, train, optimizer, criterion, 50)

Epoch: 0 	Training Loss: 10.1331847652792923497599986 	Training Accuracy: 29.1250 %
Epoch: 1 	Training Loss: 6.9644428226351742239330633 	Training Accuracy: 30.6250 %
Epoch: 2 	Training Loss: 5.9463511702418330173713912 	Training Accuracy: 30.8750 %
Epoch: 3 	Training Loss: 3.6110349155217407179918609 	Training Accuracy: 31.8750 %
Epoch: 4 	Training Loss: 1.6936505342274903451738055 	Training Accuracy: 37.1250 %
Epoch: 5 	Training Loss: 1.4109223097562790805170607 	Training Accuracy: 38.7500 %
Epoch: 6 	Training Loss: 1.3418497779965401495161359 	Training Accuracy: 42.2500 %
Epoch: 7 	Training Loss: 1.3439059898257255110110009 	Training Accuracy: 42.6250 %
Epoch: 8 	Training Loss: 1.2815954613685607288431356 	Training Accuracy: 44.7500 %
Epoch: 9 	Training Loss: 1.2844688536226749331348174 	Training Accuracy: 44.3750 %
Epoch: 10 	Training Loss: 1.2194990451633929140484724 	Training Accuracy: 48.1250 %
Epoch: 11 	Training Loss: 1.2820120908319949659670556 	Training Accuracy: 47.5000 %
E

In [15]:
train_model(50, model, train, optimizer, criterion, 75)

Epoch: 0 	Training Loss: 1.1746180090308189214454160 	Training Accuracy: 50.7500 %
Epoch: 1 	Training Loss: 1.1762165833264588421513963 	Training Accuracy: 52.6250 %
Epoch: 2 	Training Loss: 1.1776298040151595980518096 	Training Accuracy: 52.6250 %
Epoch: 3 	Training Loss: 1.1616250018775462482523153 	Training Accuracy: 53.2500 %
Epoch: 4 	Training Loss: 1.0982544814050196890775624 	Training Accuracy: 57.6250 %
Epoch: 5 	Training Loss: 1.0643252041190862655639648 	Training Accuracy: 57.7500 %
Epoch: 6 	Training Loss: 1.0964346522092818236870926 	Training Accuracy: 57.3750 %
Epoch: 7 	Training Loss: 1.1117074688524006820244949 	Training Accuracy: 58.3750 %
Epoch: 8 	Training Loss: 1.0279643197357655193258097 	Training Accuracy: 60.5000 %
Epoch: 9 	Training Loss: 0.9972056670486927343333150 	Training Accuracy: 62.5000 %
Epoch: 10 	Training Loss: 0.9231698207929730504162080 	Training Accuracy: 64.8750 %
Epoch: 11 	Training Loss: 0.9448683018237352682078267 	Training Accuracy: 65.2500 %
Ep

In [18]:
train_model(50, model, train, optimizer, criterion, 85)

Epoch: 0 	Training Loss: 0.6564196139108389171923363 	Training Accuracy: 76.6250 %
Epoch: 1 	Training Loss: 0.5866564274299890247021949 	Training Accuracy: 78.8750 %
Epoch: 2 	Training Loss: 0.5509879241907037483727549 	Training Accuracy: 78.2500 %
Epoch: 3 	Training Loss: 0.5168973928259219530190194 	Training Accuracy: 81.0000 %
Epoch: 4 	Training Loss: 0.5548173121316358757937337 	Training Accuracy: 79.2500 %
Epoch: 5 	Training Loss: 0.5312254015915095495259379 	Training Accuracy: 80.7500 %
Epoch: 6 	Training Loss: 0.4042686352436430929913058 	Training Accuracy: 85.3750 %


In [21]:
train_model(50, model, train, optimizer, criterion, 90)

Epoch: 0 	Training Loss: 0.3711949329054914370473739 	Training Accuracy: 85.7500 %
Epoch: 1 	Training Loss: 0.3573838040744885535815456 	Training Accuracy: 85.7500 %
Epoch: 2 	Training Loss: 0.3473680795845575652869286 	Training Accuracy: 86.8750 %
Epoch: 3 	Training Loss: 0.4140971993775019455163999 	Training Accuracy: 87.0000 %
Epoch: 4 	Training Loss: 0.4463953510019928461183270 	Training Accuracy: 83.1250 %
Epoch: 5 	Training Loss: 0.3327636667215847499079473 	Training Accuracy: 87.6250 %
Epoch: 6 	Training Loss: 0.3132651167665608249457421 	Training Accuracy: 88.5000 %
Epoch: 7 	Training Loss: 0.2387696057061839405299963 	Training Accuracy: 91.2500 %


In [24]:
train_model(50, model, train, optimizer, criterion, 93)

Epoch: 0 	Training Loss: 0.2571085803370806166512352 	Training Accuracy: 91.6250 %
Epoch: 1 	Training Loss: 0.2471165355409903063854671 	Training Accuracy: 90.6250 %
Epoch: 2 	Training Loss: 0.2596376840626180571014459 	Training Accuracy: 90.8750 %
Epoch: 3 	Training Loss: 0.2054060211387695733531444 	Training Accuracy: 92.8750 %
Epoch: 4 	Training Loss: 0.2576124174395954535832232 	Training Accuracy: 92.0000 %
Epoch: 5 	Training Loss: 0.2094774904733640030762842 	Training Accuracy: 93.3750 %


In [27]:
train_model(50, model, train, optimizer, criterion, 94)

Epoch: 0 	Training Loss: 0.1984503177930309891774385 	Training Accuracy: 93.2500 %
Epoch: 1 	Training Loss: 0.1655355506606429050187046 	Training Accuracy: 95.7500 %


In [30]:
train_model(50, model, train, optimizer, criterion, 96)

Epoch: 0 	Training Loss: 0.1992176318765268572263949 	Training Accuracy: 94.2500 %
Epoch: 1 	Training Loss: 0.1434537855026428720961462 	Training Accuracy: 94.7500 %
Epoch: 2 	Training Loss: 0.1756206945554004139609106 	Training Accuracy: 94.2500 %
Epoch: 3 	Training Loss: 0.1560559380337508683656011 	Training Accuracy: 93.8750 %
Epoch: 4 	Training Loss: 0.1544248796357351183328177 	Training Accuracy: 94.2500 %
Epoch: 5 	Training Loss: 0.1223846696166583508658121 	Training Accuracy: 96.0000 %


In [33]:
train_model(5, model, train, optimizer, criterion, 96)

Epoch: 0 	Training Loss: 0.1822273750941349190757279 	Training Accuracy: 93.1250 %
Epoch: 1 	Training Loss: 0.1342197854499500020430958 	Training Accuracy: 95.5000 %
Epoch: 2 	Training Loss: 0.1391250037650934301414196 	Training Accuracy: 95.1250 %
Epoch: 3 	Training Loss: 0.1624052722175110818891852 	Training Accuracy: 95.0000 %
Epoch: 4 	Training Loss: 0.0976233107599910054918979 	Training Accuracy: 96.5000 %


In [36]:
train_model(1, model, train, optimizer, criterion, 96)

Epoch: 0 	Training Loss: 0.1131080216374448360605598 	Training Accuracy: 96.3750 %


In [39]:
train_model(1, model, train, optimizer, criterion, 96)

Epoch: 0 	Training Loss: 0.0894909195468062551581667 	Training Accuracy: 97.1250 %


### Evaluate the Model on the Test Set

After training and tuning the model, evaluate it by computing its accuracy on the test set. For every image in the test set, get its prediction and compare it to its true label to validate its correctness.

In [40]:
# Calculate the accuracy of the model on the test set to evaluate its performance
def eval_model(net, testDataLoader):
    # Set the model in evaluation mode
    net.eval()
    # Initialize the counters
    correct = 0
    total = 0
    # Disable gradient descent calculation for evaluation
    with torch.no_grad():
        # Evaluate the model against the test set
        for inputs, targets in testDataLoader:
            # Get the predictions from the model on the inputs
            outputs = net(inputs)
            # Get the (value, index) tuples for the maximum values across the 1st dimension of outputs
            _, predicted = torch.max(outputs.data, 1)
            # Update the total count
            total += len(targets)
            # Compute and update the number of correct predictions by comparing against the target label
            correct += (predicted == targets).sum().item()
    
    # Compute and print the accuracy of the model on the test set
    test_acc = 100 * correct / total
    print('Test Accuracy: {:.4f} %'.format(test_acc))

In [41]:
eval_model(model, test)

Test Accuracy: 46.0000 %
